<a href="https://colab.research.google.com/github/jessalynlc/IDS705_final_project/blob/main/data_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import requests
import json
import pandas as pd
import numpy as np

api_key = 'd70a0d83e1a942578e16b2ed70dab42f'

#pulling data for ERCOT
ercot_load_forecast_dam = 'https://api.gridstatus.io/v1/datasets/ercot_load_forecast_dam/query?api_key=' + api_key + '&limit=1000'
load = 'https://api.gridstatus.io/v1/datasets/ercot_load/query?api_key=' + api_key + '&limit=1000'
ercot_unplanned_resource_outages = 'https://api.gridstatus.io/v1/datasets/ercot_unplanned_resource_outages/query?api_key=' + api_key + '&limit=1000'
ercot_fuel_mix = 'https://api.gridstatus.io/v1/datasets/ercot_fuel_mix/query?api_key=' + api_key + '&limit=1000'
ercot_solar_actual_and_forecast_hourly = 'https://api.gridstatus.io/v1/datasets/ercot_solar_actual_and_forecast_hourly/query?api_key=' + api_key + '&limit=1000'
ercot_temperature_forecast_by_weather_zone = 'https://api.gridstatus.io/v1/datasets/ercot_temperature_forecast_by_weather_zone/query?api_key=' + api_key + '&limit=1000'
ercot_energy_storage_resources = 'https://api.gridstatus.io/v1/datasets/ercot_energy_storage_resources/query?api_key=' + api_key + '&limit=1000'


#LMP price pull (target)
#Pulling LMP at the average of all the hubs
ercot_lmp_by_settlement_point = 'https://api.gridstatus.io/v1/datasets/ercot_lmp_by_settlement_point/query/location/HB_HUBAVG?api_key=' + api_key + '&limit=1000'

#Other hubs incase we want to forecast them, but sticking with the bus average:HB_CENTRAL, HB_EAST, HB_HOUSTON, HB_BUSAVG, HB_NORTH, HB_PAN, HB_SOUTH HB_WEST

#Data Pull

In [30]:
response = requests.get(ercot_load_forecast_dam)
ercot_load_forecast_dam_data = response.json()

response = requests.get(load)
load_data = response.json()

response = requests.get(ercot_unplanned_resource_outages)
ercot_unplanned_resource_outages_data = response.json()

response = requests.get(ercot_fuel_mix)
ercot_fuel_mix_data = response.json()

response = requests.get(ercot_solar_actual_and_forecast_hourly)
ercot_solar_actual_and_forecast_hourly_data = response.json()

response = requests.get(ercot_temperature_forecast_by_weather_zone)
ercot_temperature_forecast_by_weather_zone_data = response.json()

response = requests.get(ercot_energy_storage_resources)
ercot_energy_storage_resources_data = response.json()

response = requests.get(ercot_lmp_by_settlement_point)
ercot_lmp_by_settlement_point_data = response.json()

#Question: Not all the datasets start at the same time, but I almost wonder if it's interesting to see how the model performs as more data gets added as time progresses...

#Organizing data frame

In [31]:
#extracting the value of the data key to create a list of dictionaries
ercot_load_forecast_dam_records = ercot_load_forecast_dam_data['data']
load_records = load_data['data']
ercot_unplanned_resource_outages_records = ercot_unplanned_resource_outages_data['data']
ercot_fuel_mix_records = ercot_fuel_mix_data['data']
ercot_solar_actual_and_forecast_hourly_records = ercot_solar_actual_and_forecast_hourly_data['data']
ercot_temperature_forecast_by_weather_zone_records = ercot_temperature_forecast_by_weather_zone_data['data']
ercot_energy_storage_resources_records = ercot_energy_storage_resources_data['data']
ercot_lmp_by_settlement_point_records = ercot_lmp_by_settlement_point_data['data']

# Create separate data frames
ercot_load_forecast_dam_df = pd.DataFrame(ercot_load_forecast_dam_records)
load_df = pd.DataFrame(load_records)
ercot_unplanned_resource_outages_df = pd.DataFrame(ercot_unplanned_resource_outages_records)
ercot_fuel_mix_df = pd.DataFrame(ercot_fuel_mix_records)
ercot_solar_actual_and_forecast_hourly_df = pd.DataFrame(ercot_solar_actual_and_forecast_hourly_records)
ercot_temperature_forecast_by_weather_zone_df = pd.DataFrame(ercot_temperature_forecast_by_weather_zone_records)
ercot_energy_storage_resources_df = pd.DataFrame(ercot_energy_storage_resources_records)
ercot_lmp_by_settlement_point_records_df = pd.DataFrame(ercot_lmp_by_settlement_point_records)

#Combine data frames based on time
# Put all your DataFrames in a list
dataframes = [
    ercot_load_forecast_dam_df,
    load_df,
    ercot_unplanned_resource_outages_df,
    ercot_fuel_mix_df,
    ercot_solar_actual_and_forecast_hourly_df,
    ercot_temperature_forecast_by_weather_zone_df,
    ercot_energy_storage_resources_df,
    #ercot_lmp_by_settlement_point_records_df
]

#Convert timestamps to datetime
for i, df in enumerate(dataframes):
  if 'interval_start_utc' in df.columns:
    df['interval_start_utc'] = pd.to_datetime(df['interval_start_utc'])
  if 'interval_end_utc' in df.columns:
    df['interval_end_utc'] = pd.to_datetime(df['interval_end_utc'])
  dataframes[i] = df

#Find the minimum interval across all datasets
min_intervals = []
for df in dataframes:
  if 'interval_start_utc' in df.columns and 'interval_end_utc' in df.columns:
    deltas = (df['interval_end_utc'] - df['interval_start_utc']).dropna()
    if not deltas.empty:
      min_intervals.append(deltas.min())

# Determine smallest common time delta
smallest_interval = min(min_intervals)
print(f"Smallest interval: {smallest_interval}")

#Build master time index from earliest to latest across all data
all_times = pd.concat([df['interval_start_utc'] for df in dataframes if 'interval_start_utc' in df.columns])
time_index = pd.date_range(start=all_times.min(), end=all_times.max(), freq=smallest_interval)

# Create the master DataFrame to merge onto
master_df = pd.DataFrame({'interval_start_utc': time_index})

#Merge each DataFrame onto master
for df in dataframes:
    if 'interval_start_utc' in df.columns:
        master_df = master_df.merge(df, on='interval_start_utc', how='left')

print(master_df.head())
print(master_df.shape)

KeyError: 'data'